# KB데이터

![](kb.png) 60대 이상 소비자를 타켓으로 어느 지역을 활성화하는게 효과적일까?

### 

## 

# 라이브러리 불러오기


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## 

# 데이터 불러오기 :

# pop 데이터의 문자열 ','를 제외하고 수치컬럼으로 가져오기


In [ ]:
market_raw = pd.read_excel("data/traditionalmarket.xlsx")
market = market_raw.copy()
pop_raw = pd.read_excel('data/pop_2015.xlsx', thousands = ',') #쉼표 없애주고 숫자로 바꿔줌.
pop = pop_raw.copy()

## 

# 데이터 전처리 1 :

# pop 데이터 필요한 부분만, index 재부여


In [ ]:
pop = pop.iloc[1:18,[0,1,3,4,5,6,7,8,9,10,11,12,13]]
pop.head()

In [ ]:
pop = pop.reset_index().iloc[:,1:]  # 인덱스를 0부터 다시 부여
pop.head()

## 

# 변수명 변경


In [ ]:
market = market.rename(columns={'시장명' : 'market_name',
                              '시장유형' : 'type',
                              '소재지도로명주소' : 'address_road',
                              '시장개설주기' : 'open_period',
                              '소재지지번주소' : 'address_old',
                              '점포수' : 'store_count',
                              '사용가능상품권' : 'certificate',
                              '공중화장실 보유여부' : 'public_toilet',
                              '주차장 보유여부' : 'parking_lot',
                              '개설년도' : 'year' ,
                              '데이터기준일자' : 'data_date'})

pop = pop.rename(columns={'행정기관' : 'region',
                          '총 인구수' : 'total_pop',
                          '0~9세' : '0~9',
                          '10~19세' : '10~19',
                          '20~29세' : '20~29',
                          '30~39세' : '30~39',
                          '40~49세' : '40~49',
                          '50~59세' : '50~59',
                          '60~69세' : '60~69',
                          '70~79세' : '70~79',
                          '80~89세' : '80~89',
                          '90~99세' : '90~99',
                          '100세 이상' : '100_over',
                          })

## 

# 데이터 전처리 2 :

# pop 데이터의 모든 수치 컬럼을 정수 변환하기


In [ ]:
pop.head()

total_pop 컬럼이 문자열 컬럼이다. 그 외 컬럼들은 불필요한 실수값이다.


In [ ]:
region = pop['region'] #행정기관 빼내기
pop = pop.iloc[:,1:].astype(int) #행정기관 빼고 나머지 열을 정수로 바꿈.
pop = pd.concat([region, pop], axis = 1) #빼낸 행정기관을 다시 합침
pop.info()

## 

# 데이터 전처리 3 :

# market 데이터의 주소 컬럼에서 지역명 뽑아오기


In [ ]:
market['address_road'].head()  # 도로주소 컬럼 확인

In [ ]:
market['region'] = market["address_road"].str.split(' ').str[0]  # 공백을 기준으로 분리함.

# 데이터 전처리 4 :

# pop 데이터 지역명을 market 데이터와 동일하게 수정


In [ ]:
pop.head()  # pop 데이터는 region 컬럼을 가지고 있음.

In [ ]:
pop['region'] = pop['region'].str.replace('세종특별자치시','세종특별시')  # 값 바꾸기
pop['region'] = pop['region'].str.replace('제주특별자치도','제주도')

In [ ]:
pop['region'][0]

In [ ]:
pop['region'] = pop['region'].str.replace('  ','')  # 공백 없애기

## 

# 데이터 전처리 5 :

# 인구 연령 범주화 하기


In [ ]:
pop.head()  # pop 데이터 확인

In [ ]:
# 연령 구간별 범주화
pop['under20'] = pop['0~9'] + pop['10~19']
pop['2050'] = pop['20~29'] + pop['30~39'] + pop['40~49'] + pop['50~59']
pop['over60'] = pop['60~69'] + pop['70~79'] + pop['80~89'] + pop['90~99'] + pop['100_over']

## 

## 

# 지역별 데이터 프레임 만들기


In [ ]:
market_region = market.groupby('region', as_index = False)\
      .agg(market_count = ('market_name', 'count'),
           store_sum = ('store_count', 'sum'), 
           certificate_count = ('certificate', 'count'))

In [ ]:
pop_region = pop[['region', 'total_pop', 'under20', '2050', 'over60']]

market_pop = pd.merge(market_region, pop_region, how='left', on='region')

## 

# 지역별 데이터 프레임에 파생 변수 추가하기 1

## (연령 구간별 인구 비율 계산)


In [ ]:
market_pop['pct_under20'] = round(market_pop['under20'] / market_pop['total_pop'] * 100, 1) #미성년자 비율 추가 + 소숫점 정리
market_pop['pct_2050'] = round(market_pop['2050'] / market_pop['total_pop'] * 100, 1)
market_pop['pct_over60'] = round(market_pop['over60'] / market_pop['total_pop'] * 100, 1)

## 

# 지역별 데이터 프레임에 파생 변수 추가하기 2

## (시장.점포 1개 당 60대 이상 인구 비율 계산)


In [ ]:
market_pop['old_per_market'] = round(market_pop['over60'] / market_pop['market_count'], 1)
market_pop['old_per_store'] = round(market_pop['over60'] / market_pop['store_sum'], 1)

## 

# 지역별 데이터 프레임에 파생 변수 추가하기 3

## (60대 이상 인구 비율이 평균보다 큰 지역, 작은 지역 구분)


In [ ]:
# 60대 이상 비율의 평균을 구한 뒤 그것보다 높거나 낮다는것 표시.
market_pop['over60_mean_updown'] = \
  np.where(market_pop['pct_over60'] >= market_pop['pct_over60'].mean(), 'UP', 'DOWN') 

## 평균 기준으로 한 근거 : 60대 이상 박스플롯


In [ ]:
px.box(data_frame = market_pop, y = 'pct_over60')

## 

# 지역별 데이터 프레임에 파생 변수 추가하기 4

## (온누리 상품권을 사용하는 시장 수)


In [ ]:
market_pop['pct_certificate'] = \
 round(market_pop['certificate_count'] / market_pop['market_count'] *100, 1)

In [ ]:
market_pop.head()

## 

## 

# 시각화 1 : 지역별 인구 수 막대 그래프


In [ ]:
px.bar(data_frame = market_pop,  x='total_pop', y='region', orientation='h', color='region') \
  .update_layout(yaxis = {"categoryorder":"total ascending"}).show()

경기도와 서울이 가장 많다.

## 

# 시각화 2 : 지역별 60세 이상 인구 비율 막대 그래프


In [ ]:
 px.bar(data_frame = market_pop, x='over60', y='region',  orientation='h', color ='region') \
  .update_layout(yaxis = {"categoryorder":"total ascending"}).show()

경기도와 서울이 가장 많다.

## 

# 시각화 3 : 지역별 인구 비율 누적 막대 그래프

#### 보고 싶은 데이터만 df로 가져옴


In [ ]:
df = market_pop[['region', 'pct_under20', 'pct_2050', 'pct_over60']].sort_values('pct_over60', ascending = False)
df

#### 데이터프레임을 길게 변환


In [ ]:
df_melted = df.melt(id_vars='region', var_name='ageg', value_name='pct')
df_melted.head(20)

In [ ]:
# Plotly Express를 사용하여 누적 가로 막대 그래프 생성
fig = px.bar(df_melted,          # 데이터프레임 df_melted를 사용
             x='pct',            # x축에는 'pct(각 연령대의 비율)' 열을 배치.
             y='region',         # y축에는 'region(각 지역)' 열을 배치.
             color='ageg',       # 'ageg' 열에 따라 막대의 색상을 지정합니다.
             orientation='h',    # 가로 방향으로 막대 그래프를 생성합니다.
             color_discrete_map={'pct_under20': 'skyblue',
                                 'pct_2050': 'lightyellow',
                                 'pct_over60': 'salmon'},     # 'ageg' 열의 값: '색상' 딕셔너리 매핑하여 색상을 지정합니다.
             category_orders={'region': df['region'].tolist()})  # df의 region을 기준으로 그래프 정렬 

# 그래프 설정
fig.update_layout(
    barmode='stack',     # 누적 막대 그래프를 생성합니다.
    title='연령대별 인구 비율 비교',  # 그래프의 제목을 설정합니다.
    xaxis_title='pct',   # x축의 레이블을 설정합니다.
    yaxis_title='region'  # y축의 레이블을 설정합니다.
)

fig.show()  # 그래프를 출력합니다.

## 

# 시각화 4 : 지역별 시장 수 막대그래프


In [ ]:
px.bar(data_frame = market_pop,  x='market_count', y='region', orientation='h', color='region') \
  .update_layout(yaxis = {"categoryorder":"total ascending"}).show()

서울, 부산, 경남, 경북, 경기도 순으로 시장 수가 많다.

## 

# 시각화 5 : 지역별 점포 수 막대 그래프


In [ ]:
px.bar(data_frame = market_pop,  x = 'store_sum', y = 'region', orientation='h', color = 'region') \
  .update_layout(yaxis = {"categoryorder":"total ascending"}).show()

서울, 부산, 경기도, 경남, 경북 순으로 시장 수가 많다.

## 

# 시각화 6 : 지역별 60대 인구 / 시장 수


In [ ]:
px.bar(data_frame = market_pop, x = 'old_per_market', y = 'region', orientation ='h', color = 'region') \
 .update_layout(yaxis = {"categoryorder":"total ascending"}).show()

경기도가 가장 많음.

## 

# 시각화 7 : 지역별 60대 인구 / 점포 수


In [ ]:
px.bar(data_frame = market_pop, x = 'old_per_store', y = 'region', orientation = 'h', color = 'region') \
  .update_layout(yaxis = {"categoryorder":"total ascending"}).show()

경기도가 가장 많음.

## 

# 경기도의 온누리 상품권을 사용하는 시장 수


In [ ]:
market_pop.query('region == "경기도"')["pct_certificate"]